In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warning 메시지 제거, warning 메시지는 라이브러리 업데이트나 사용법에 대한 안내 등이 있습니다.
# 코딩을 처음 시작할 때는  warning 메시지가 나오면 당황하실 수도 있어서 제거를 하고 보도록 합니다.
# warning 메시지는 제거하고 보셔도 되지만 Error 메시지를 꼭 고쳐주셔야 합니다.
warnings.filterwarnings('ignore', 'This pattern has match groups')
warnings.filterwarnings('ignore', 'The iterable function was deprecated in Matplotlib')

In [5]:
import pandas as pd
import numpy as np
# 기존 plotnine(ggplot)코드를 pandas와 seaborn으로 변경
# from plotnine import *
import seaborn as sns
# 지도 시각화를 위해
import folium

# 노트북 안에서 그래프를 표시하기 위해
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('font', family='gulim')

여기서 소스 보고 학습할 것
https://github.com/corazzon/OpenDataWrangling/blob/master/store_location_by_folium.ipynb

In [6]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
# Window 의 한글 폰트 설정
# plt.rc('font',family='gulim')
# Mac 의 한글 폰트 설정
plt.rc('font', family='gulim') 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

In [28]:
shop_2018 = pd.read_csv('../testData/shop_201912_01.csv', encoding='cp949')
#shop_2018.head()

In [ ]:
shop_2018_2.head(3)

In [ ]:
coffee = shop_2018[shop_2018['상권업종소분류명'].str.contains('커피')]
coffee.shape

In [ ]:
coffee.loc[coffee['상호명'].str.contains('스타벅스'), '상호명'].unique()

In [ ]:
coffee['시도명'].isnull().sum()

In [ ]:
# 지난 데이터와 마찬가지로 서울과 부산만 있습니다.
# 다른 지역을 참고하고 싶다면 같은 폴더에 있는 다른 파일을 참고해 주세요.
coffee['시도명'].value_counts()

In [ ]:
shop_2018[:1000].plot.scatter(x="경도", y="위도", grid="True")

In [ ]:
shop_2018.columns

In [29]:
df_seoul = shop_2018.loc[shop_2018['시도명'].str.startswith('서울')].copy()
df_seoul.shape

(407376, 39)

In [ ]:
df_seoul['상권업종대분류명'].value_counts()

In [ ]:
df_seoul.describe(include=np.object) # oject 변수 개수

In [ ]:
df_seoul[['위도', '경도']].describe(include=np.number)

In [ ]:
sns.countplot(data=df_seoul, y="상권업종대분류명")

## 베스킨,던킨 분석

In [30]:
df_31 = df_seoul[df_seoul['상호명'].str.contains('배스킨|던킨')].copy()
df_31.shape

(537, 39)

In [ ]:
df_31.columns

In [31]:
df_31 = df_31[['상호명', '지점명', '상권업종대분류명', '상권업종중분류명', 
               '지번주소', '도로명주소',  '위도', '경도', '시군구명', '행정동명']].copy()

In [32]:
df_31.loc[df_31['상호명'].str.contains('배스킨'), '브랜드명'] = '배스킨라빈스'
df_31.loc[df_31['상호명'].str.contains('던킨'), '브랜드명'] = '던킨도너츠'

In [ ]:
df_31.브랜드명.value_counts()

In [33]:
df_31['위도'] = df_31['위도'].astype(float)
df_31['경도'] = df_31['경도'].astype(float)

In [ ]:
df_31.plot.scatter(x='위도',y='경도')

In [ ]:
# seaborn 으로 scatter plot 그리기
sns.scatterplot(data=df_31, x='경도', y='위도', hue="브랜드명")

### folium 사용

In [34]:
geo_df = df_31.copy()
# 한글 폰트 깨지는 것을 막기위한 처리
#import platform
#from matplotlib import font_manager, rc

##if platform.system() == 'Windows':
#font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/gulim.ttc").get_name()
font_name ='gulim'
plt.rc('font', family=font_name)

# 지도를 초기화 해줄 때 어디를 중심으로 보여줄지 설정합니다.
# 우리가 가져온 데이터프레임 안에 있는 데이터를 기준으로 출력할 수 있도록 위경도의 평균값을 구해옵니다.
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], zoom_start=12)

marker='normal'
for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '브랜드명']  + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df['브랜드명'][n] == '던킨도너츠' :
        icon_color = 'pink'
    else:
        icon_color = 'blue'    
    lat=geo_df.loc[n, '위도']
    long=geo_df.loc[n, '경도']
    if marker=='normal' :
        folium.Marker([lat, long], 
                  popup=popup_name, 
                  icon=folium.Icon(color=icon_color)
        ).add_to(map)
    
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다.
    else: 
        folium.CircleMarker(
            location=[lat, long],
            radius=3,     #마커의 반지름 크기  지정->가변적으로 가능함
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)

map

# 지도를 불러올 때 주피터 노트북에서 iframe으로 불러오는 데 이 때 인코딩이 깨질수가 있습니다.
# html로 저장후 확인해보면 알수 있음
# map.save('index.html')

## 스타벅스/이티야

In [ ]:
df_incheon = shop_2018_2.loc[shop_2018_2['시도명'].str.startswith('인천')].copy()

In [ ]:
df_cafe=pd.concat([df_incheon,df_seoul])

In [ ]:
df_cafe.시도명.value_counts()

In [ ]:

df_cafe = df_cafe[df_cafe['상호명'].str.contains('스타벅스|starbucks|STARBUCKS|이디야|ediya|EDIYA')]


In [ ]:
df_cafe.loc[df_cafe['상호명'].str.contains('스타벅스|starbucks|STARBUCKS'),'브랜드명']='스타벅스'

In [ ]:
df_cafe['브랜드명']=df_cafe.브랜드명.fillna('이디야')

In [ ]:
df_cafe['브랜드명'].value_counts()

In [ ]:
geo_df = df_cafe.copy()
font_name ='gulim'
plt.rc('font', family=font_name)

map = folium.Map(location=[geo_df['위도'].mean(),geo_df['경도'].mean()], zoom_start=12,
                 tiles='Stamen Toner')


for n in geo_df.index:
    popup_name = geo_df.loc[n, '브랜드명']  + ' - ' + geo_df.loc[n, '도로명주소']
    if geo_df['브랜드명'][n] == '스타벅스' :
        icon_color = 'pink'
    else:
        icon_color = 'blue'    
    lat=geo_df.loc[n, '위도']
    long=geo_df.loc[n, '경도']
    folium.CircleMarker(
        location=[lat, long],
        radius=3,    
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)

map


In [ ]:
df_cafe_brand=df_cafe.groupby(['시군구명','브랜드명'])['상호명'].count().reset_index()
df_cafe_brand.columns = ['구', '브랜드명', '매장수']
df_cafe_brand

In [ ]:
plt.figure(figsize=(24, 6))
sns.barplot(data=df_cafe_brand, x='구', y='매장수', hue='브랜드명')

In [ ]:
df_cafe_brand

## choropleth 그리기  
choropleth는 시군 등 단위지역(unit area)으로 구성된 지역의 통계량의 크기를 부호로 나타내는 지도이다.  
구별로 매장수를 표현하기 위해 GeoJSON 파일 로드  
파일출처 : southkorea/seoul-maps: Seoul administrative divisions in ESRI Shapefile, GeoJSON and TopoJSON formats. 

지도 출력 시 색상 참고
http://colorbrewer2.org/

In [ ]:
df_cafe_gu = pd.DataFrame(df_cafe.groupby(['시군구명'])['상호명'].count())
df_cafe_gu.rename(columns={'상호명':'매장수'},inplace=True)
df_cafe_gu.head()

### 스타벅스,이디야 매장분포

In [ ]:
import json

geo_path = '../testData/seoul_municipalities_geo_simple.json' #서울
geo_path = '../testData/skorea_municipalities_geo_simple.json' #전국
geo_json = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
d1=df_cafe_gu.reset_index()
d2=df_cafe_starbucks.reset_index()
set(d1.시군구명).difference(set(d2.시군구명))  ##차집합

In [ ]:
df_cafe_gu

In [ ]:
geo_df = df_cafe.copy()

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11)

#choropleth: 단위지역(unit area)으로 구성된 지역의 통계량의 크기를 부호로 나타내는 지도

map.choropleth(geo_data = geo_json,  #시군구 경계 json
               data = df_cafe_gu['매장수'],  #데이터
               columns = [df_cafe_brand.index, df_cafe_gu['매장수']],
               fill_color = 'YlGnBu',
               key_on = 'feature.properties.name',  #json파일의 시군구명key name
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'    
    
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다. 
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)

map

## 스타벅스 매장분포

In [ ]:
df_cafe_s=df_cafe[df_cafe.브랜드명=='스타벅스']

In [ ]:
df_cafe_starbucks=pd.DataFrame(df_cafe_s.groupby(['시군구명'])['상호명'].count())
df_cafe_starbucks.columns=['매장수']

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_starbucks['매장수'],
               columns = [df_cafe_starbucks.index, df_cafe_starbucks['매장수']],
               fill_color = 'YlGnBu',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'    

            
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

##이디야 매장분포

In [ ]:
df_cafe_temp = df_cafe[df_cafe['브랜드명'] == '이디야']
df_cafe_ediya = pd.DataFrame(df_cafe_temp.groupby(['시군구명'])['상호명'].count())
df_cafe_ediya.columns = ['매장수']
df_cafe_ediya.head()

In [ ]:
ap = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_ediya['매장수'],
               columns = [df_cafe_ediya.index, df_cafe_ediya['매장수']],
               fill_color = 'PuBu',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '이디야' :
        icon_color = 'blue'    

            
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

## 매장수 비교

In [ ]:
df_cafe_brand_vs = df_cafe_brand.pivot_table(index='구',
                                             columns='브랜드명',
                                             values='매장수')
df_cafe_brand_vs.columns = ['스타벅스', '이디야']
df_cafe_brand_vs.head(2)

In [ ]:
df_cafe_brand.head(2)

In [ ]:
df_cafe_brand_vs['매장수비교'] = df_cafe_brand_vs.apply( 
    lambda x : 1 if x['스타벅스'] > x['이디야'] else 0, axis=1)
df_cafe_brand_vs.head()

In [ ]:
lng_list = []
lat_list = []
for gu in df_cafe_brand_vs.index:
    lat = df_cafe.loc[df_cafe['시군구명'] == gu, '위도'].mean()
    lng = df_cafe.loc[df_cafe['시군구명'] == gu, '경도'].mean()
    lat_list.append(lat)
    lng_list.append(lng)
    
df_cafe_brand_vs['위도'] = lat_list
df_cafe_brand_vs['경도'] = lng_list
df_cafe_brand_vs.head()

In [ ]:
df_cafe_brand_vs['스타벅스'] = df_cafe_brand_vs['스타벅스'].astype(float)
df_cafe_brand_vs['이디야'] = df_cafe_brand_vs['이디야'].astype(float)
df_cafe_brand_vs.info()

In [ ]:
df_cafe_brand_vs

In [ ]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_brand_vs['매장수비교'],
               columns = [df_cafe_brand_vs.index, 
                          df_cafe_brand_vs['매장수비교']],
               fill_color = 'BuGn',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for gu in df_cafe_brand_vs.index:
        
    for cafe in ['스타벅스', '이디야']:
        
        cafe_count = df_cafe_brand_vs.loc[gu, cafe]
        msg = f'{gu} {cafe} 매장수 : {cafe_count:.0f}'
        
        icon_color = 'blue'
        if cafe == '스타벅스': 
            icon_color = 'green'
        
        
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location = [df_cafe_brand_vs.loc[gu,'위도'], 
                        df_cafe_brand_vs.loc[gu, '경도']],
            radius = cafe_count,
            color = icon_color,
            popup = msg,
            fill = True,
            fill_color = icon_color
        ).add_to(map)

map

In [ ]:
df_cafe_brand_vs